# Redes Neuronales Artificiales
## Perceptrón

El preceptrón es una estructura que trata de imitar el funcionamiento de una neurona.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Blausen_0657_MultipolarNeuron.png/1024px-Blausen_0657_MultipolarNeuron.png" alt="Neurona" style="width: 400px;"/>

> Fig. 1: [Imágen Neurona Wikipedia](https://en.wikipedia.org/wiki/Neuron) <br>

Donde el modelo se simplifica a: <br>

<img src="https://upload.wikimedia.org/wikipedia/commons/3/31/Perceptron.svg" alt="Perceptrón" style="width: 400px;"/>

> Fig. 2: [Imágen Perceptrón Wikipedia](https://en.wikipedia.org/wiki/Perceptron) <br>

Considerando $\overline{x}=(x_{1}, x_{2},...,x_{n})$ y $\overline{w}=(w_{1}, w_{2},...,w_{n})$: <br>
$$percept(\overline{x})=f(\overline{x} \cdot \overline{w} + b)=f(\sum(x_{i} \cdot x_{i} )+ b)$$
Dependiendo como se seleccione $f(o)$, un perceptrón es simirar a una regresión lineal ($f(o)=o$) o a una regresión lógistica $f(o)=\frac{1}{1+e^{-o}}$. Obviamente, existen otras funciones de activación que se irán discutiendo a lo largo del curso.

Combinado diversos preceptrones en forma paralela se forma una capa de una red neuronal. En este caso, la capa se conoce como una capa densa, ya que todas las salidas de la capa están conectados con cada entrada. En el caso de una capa, $W$ es una matriz de dimensiones $cantidad\ de\ características$ X $cantidad\ de\ perceptrones$, $b$ es un vector de dimensionalidad $cantidad\ de\ perceptrones$, y $f(o)$ se aplíca elemento a elemento del resultado. Considerando el trabajo práctico de la clase anterior $W$ es una matriz de 786 X 10, y $b$ es un vector de 10 elementos.

## Keras
[Keras](https://keras.io/) en una librería de alto nivel para redes neuronales que abstrae las operaciones más comunes de las redes neuronales facilitando la escritura de un código más limpio. Keras soporta diversos backend para realizar las operaciones aritméticas, por ejemplo [TensorFlow](https://keras.io/backend/), [Theano](http://deeplearning.net/software/theano/) y [CNTK](https://www.microsoft.com/en-us/cognitive-toolkit/).
A continuación se muesta un ejemplo de utilización de Keras para el problema clasificación de MNIST.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from keras.layers import Input, Dense
from keras.models import Model 
from keras.utils import to_categorical

print('Cargando el dataset')
(x_train, y_train), (x_test, y_test) = mnist.load_data()
size = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], size)) / 255
x_test = x_test.reshape((x_test.shape[0], size)) / 255

In [ ]:
#Inicializo la entrada como un vector de tamaño size
i = Input(shape=(size,)) 
#Inicializo una capa densa, activación sigmoide y entrada i
#Para inicializar la capa densa se usa la API funcional de keras
d = Dense(10, activation='sigmoid')(i) 
#Inicializo el modelo a partir de sus entradas y salidas
model = Model(inputs=i, outputs=d)
#Compilo el modelo con la función de pedidad y utilizando 
#Stocastic Gradiant Descent como optimizador (una variante del Gradient Descent)
#metrics no es necesario, pero permite usar otra función de error para la validación
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])
#Muestro la esturctura del perceptrón
model.summary()
#Entreno y guardo el historial del errores en h. 
#verbose: 0: sin salida, 1: salida detallada, 2: salida solo al final del epoch
h = model.fit(x_train, to_categorical(y_train), 
              batch_size=1000, epochs=100, 
              validation_data=(x_test, to_categorical(y_test)), verbose=2)

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy del clasificador en validación: {}'.format(
        accuracy_score(y_test, 
                       np.argmax(model.predict(x_test), axis=1))))

print('Valor del error por epoch')

v_loss = h.history['val_loss']
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.plot(range(len(v_loss)), v_loss, '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

print('Valor del accuracy por epoch')

plt.plot(range(len(v_loss)), h.history['categorical_accuracy'], '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

## Stocastic Gradient Descent
Hasta ahora estabamos usando _gradient descent_ en el cual se calculaba el gradiente de la función de error sobre todas las instancias de entrenamiento. Esto puede ser muy costoso o impracticable si el conjunto de datos de entrenamiento es muy grande. Para solucionar este problema se usa el _Stocastic Gradient Descent_, que intenta aproximar el gradiente de forma aleatoria.

```
w = parámetros_a_entrenar()
x, y = dataset_entrenaminto()
for e in range(epochs):
    aleatorizar(x, y)
    for x_a, y_a in separar_en_batchs(x, y, tamaño_batch):
        g = calcular_gradiente(perdidad(x, y, w), w)
        w = w - lr * g

```
Además, como la actualización se hace más seguido, es posible usar un `lr` menor mejorando la búsqueda. Por defecto, Keras inicializa el `lr=0.01` para el _Stocastic Gradient Descent_. Es importante notar que si el tamaño del batch es igual a la cantidad de elementos en el conjunto de entrenamiento el _Stocastic Gradient Descent_ es efectivamente un _Gradient Descent_.

In [ ]:
#Importa clases necesarias para la prueba.
#Si Google Colab falla al importar agregar la linea "!pip install tqdm" sin las comillas
#para instalar el paquete. Es una barra de progreso
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.optimizers import SGD
#Inicializa una regresión lógistica
i = Input(shape=(size,)) 
d = Dense(10, activation='sigmoid')(i) 
model = Model(inputs=i, outputs=d)
#Uso SGD con un lr de 0.1 para mostrar mejor los efectos.
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['categorical_accuracy'])
#Paso y_train e y_test al formato categórico
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

#Configuración de la prueba
batch_size = 10
cant = 100
sample = 1

error_train_batch = []
error_test_batch = []

#Esto se haría dentro de un epoch
xt, yt = shuffle(x_train, y_train_cat)
indexes = range(0, xt.shape[0], batch_size)

indexes = indexes[:min(cant, len(indexes))] # Esto es solo para limitar la cantidad de pruebas
for i, j in tqdm(list(enumerate(indexes))):
    model.fit(xt[j:min(j + batch_size, yt.shape[0]), :], 
              yt[j:min(j + batch_size, yt.shape[0]), :], 
              batch_size=batch_size, epochs=1, 
              verbose=0)
    #Cada "sample" batches tomo una medición de los errores
    if i % sample == 0:
        error_train_batch.append(model.evaluate(x_train, y_train_cat, verbose=0)[0])
        error_test_batch.append(model.evaluate(x_test, y_test_cat, verbose=0)[0])
#Imprimo el error por batch
print('Valor del error por batch')

plt.plot(range(0, sample * len(error_train_batch), sample), error_train_batch, '-r')
plt.plot(range(0, sample * len(error_train_batch), sample), error_test_batch, '-b')
plt.xlabel('Batch')
plt.ylabel('Error')
plt.show()


### Otras variaciones del SGD
También se suele usar un factor de momento para hacer que esta curva de error sea más suavizada. La efectividad del uso de momento depende del dataset. A continuación se preseta el algoritmo modificado. `v` representa la velocidad de cambio, `m` es el parámetro que asigna importancia a la velocidad de cambio anterior (generalmente se utiliza 0.9).
```
w = parámetros_a_entrenar()
v = ceros_como(w)
x, y = dataset_entrenaminto()
for e in range(epochs):
    aleatorizar(x, y)
    for x_a, y_a in separar_en_batchs(x, y, tamaño_batch):
        g = calcular_gradiente(perdidad(x, y, w), w)
        v = momento * v - lr * g
        w = w + v

```
Existen más variaciones sobre la técnicas basadas en gradiente. Si bien se irán cubriendo este tema durante el curso, se recomienda la lectura de [An overview of gradient descent optimization algorithms](https://arxiv.org/abs/1609.04747). En particular, la sección 4. Más información sobre este y otros optimizadores se encuentra disponible en la documentación de [los optimizadores de Keras](https://keras.io/optimizers/)
## Ejercicio
Utiliza batch de 60000 para utilizar el _Stocastic Gradient Descent_ como _Gradient Descent_. Pruebe con 100 epochs por cuestiones de tiempo. 

1. ¿Qué sucede cuando se utiliza el `lr` por defecto?
1. ¿Qué sucede cuando se incrementa el `lr` a 0.1?
1. ¿Qué sucede cuando se incrementa el `lr` a 1?

_Opcional_: Conteste las mismas preguntas con más epochs, por ejemplo 1000 o 10000. 

In [ ]:
#Escriba su código aquí.

# Softmax y Categorical_crossentropy
El problema de NMIST es un problema de clasificación múlticlases o categórico, donde una imagen representa un y solo un dígito. Sin embargo, cuando entrenamos la regresión logística no hacemos uso de ese hecho, tanto las función de activación sigmoide como la función de error _binary crossentropy_ (equivalente a la entropía cruzada que definimos en el práctico anterior) trabajan cada elemento de las predicciones de forma independiente.
## Softmax
Softmax es una función de activación que transforma la salida de la regresión a una distribución de probabilidades. Es decir, la suma de cada vector de predicciones es 1 y el valor de cada elemento es la probabilidad de que la instancia clasificada pertenezca a esa clase.

$$softmax(o)_j=\frac{e^{o_j}}{\sum e^{o_i}}$$

Esta función se aplica a cada elemento, pero conoce los otros elementos del vector.

In [ ]:
i = Input(shape=(size,)) 
d = Dense(10, activation='softmax')(i) 
model = Model(inputs=i, outputs=d)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])

print('Entrenando con softmax')
h = model.fit(x_train, to_categorical(y_train), 
              batch_size=1000, epochs=100, 
              validation_data=(x_test, to_categorical(y_test)), verbose=0)
print('Accuracy del clasificador en validación: {}'.format(
        accuracy_score(y_test, 
                       np.argmax(model.predict(x_test), axis=1))))

print('Valor del error por epoch')

v_loss = h.history['val_loss']
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.plot(range(len(v_loss)), v_loss, '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

print('Valor del accuracy por epoch')

plt.plot(range(len(v_loss)), h.history['categorical_accuracy'], '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

## Categorical Crossentropy
Esta función de error considera el error sobre la categoría real, normalizando el valor de la predicción. Considerando $\hat{y}=(\hat{y}_1, \hat{y}_2, ..., \hat{y}_C)$ in vector de valores asociados a las clases
$$P_\hat{y}=\frac{\hat{y}}{\sum\hat{y}_i}$$
$$CCE(y,P_\hat{y})=-\frac{\sum y * log(P_\hat{y})}{N} $$
Notese que el valor de error se considera solo sobre las clases verdaderas, las otras son afectadas a través de la normalización de la salida $\hat{y}$.

In [ ]:
i = Input(shape=(size,)) 
d = Dense(10, activation='sigmoid')(i) 
model = Model(inputs=i, outputs=d)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])

print('Entrenando con softmax')
h = model.fit(x_train, to_categorical(y_train), 
              batch_size=1000, epochs=100, 
              validation_data=(x_test, to_categorical(y_test)), verbose=0)
print('Accuracy del clasificador en validación: {}'.format(
        accuracy_score(y_test, 
                       np.argmax(model.predict(x_test), axis=1))))

print('Valor del error por epoch')

v_loss = h.history['val_loss']
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.plot(range(len(v_loss)), v_loss, '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

print('Valor del accuracy por epoch')

plt.plot(range(len(v_loss)), h.history['categorical_accuracy'], '-b')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

In [ ]:
#Escriba código para entrenar una salida con softmax y categorical_crossentropy

## Red neuronal: Perceptrón multi-capa
La red neuronal más sencilla, conocida como perceptrón multi-capa, no es más que capas de preceptrones aplicadas una sobre la otra.

Entonces un preceptron multi-capa tiene la siguiente forma:

$$l_{1}=f_{1}(\overline{x} \cdot W_{1} + \overline{bias_{1}})$$

$$l_{2}=f_{2}(\overline{l_{1}} \cdot W_{2} + \overline{bias_{2}})$$

$$...$$

$$l_{N}=f_{N}(\overline{l_{N-1}} \cdot W_{N} + \overline{bias_{N}})$$

Es importante destacar que, dado una función de error, calcular el gradiente para cada parámetro de la red, sea $W_{i}$ o $bias_{i}$, es simplemente aplicar la regla de la cadena en repetidas oscaciones.

Uno de los motivos para la existencia de los preceptrones multicapas, es que no todos los problemas se pueden resolver con funciones aplicadas sobre operaciones lineales. Por ejemplo, no se puede hacer una regresión logística (ni lineal) que aproxime la función XOR. Considerando el dataset:

| $X_0$ | $X_1$ | $Y$ |
| --- | --- | --- |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [ ]:
x = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.asarray([[0], [1], [1], [0]])
i = Input(shape=(2,)) 
d = Dense(1, activation='sigmoid')(i) 
model = Model(inputs=i, outputs=d)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['binary_accuracy'])
h = model.fit(x, y, batch_size=1, epochs=10000, verbose=0)
print(model.predict(x))

print('Valor del error por epoch')
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

Pero al agregar una nueva capa:

In [ ]:
i = Input(shape=(2,)) 
d = Dense(8, activation='sigmoid')(i) 
d = Dense(1, activation='sigmoid')(d)
model = Model(inputs=i, outputs=d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['binary_accuracy'])
h = model.fit(x, y, batch_size=1, epochs=10000, verbose=0)
print(model.predict(x))

print('Valor del error por epoch')
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

De hecho, se ha demostrados [1] que $F(X)=\sum v_i * sigmoid(X \dot W)$, es decir una red con una sola capa oculta, puede aproximar cualquier funcion $f(X)$. Sin embargo, la prueba no dice nada de cuantas neuronas ocultas se necesitan, ni de como entrenar los parámetros.

[1] Cybenko, G. (1989)  "[Approximations by superpositions of sigmoidal functions](https://doi.org/10.1007%2FBF02551274)", Mathematics of Control, Signals, and Systems, 2(4), 303-314.

In [ ]:
i = Input(shape=(2,)) 
d = Dense(8, activation='tanh')(i) 
d = Dense(1, activation='sigmoid')(d)
model = Model(inputs=i, outputs=d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['binary_accuracy'])
h = model.fit(x, y, batch_size=1, epochs=1000, verbose=0)
print(model.predict(x))

print('Valor del error por epoch')
loss = h.history['loss']
plt.plot(range(len(loss)), loss, '-r')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

## Trabajo práctico
Implemente una red con más de una capa para resolver el problema de MNIST. Consideraciones:
* Utilice la función de activación softmax para la última capa y categorical cross entropy como función de error.
* Pruebe diversas cantidad de capas ocultas (entre 1 y 4).
* Pruebe distintas funciones de activación para la capa intermedia. Por ejemplo: sigmoide, tangente hiperbólica, relu, o lineal. Puede encontrar más información sobre las funciones de activación en [Keras](https://keras.io/activations/).

In [ ]:
#Escríba su código aquí.